In [14]:
import numpy as np
from scipy import sparse
import pandas as pd

#### Read data

In [87]:
wiki = pd.DataFrame.from_csv('people_wiki.csv')

In [88]:
wiki = wiki.reset_index()

In [89]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return sparse.csr_matrix( (data, indices, indptr), shape)
    
corpus = load_sparse_csr('people_wiki_tf_idf.npz')

In [90]:
map_index_to_word = pd.read_json('people_wiki_map_index_to_word.json', typ='series', orient='records')

#### Generate a collection of random vectors

In [91]:
def generate_random_vectors(num_vector, dim):
    return np.random.randn(dim, num_vector)

Generate 3 random vectors of dimension 5, arranged into a single 5 x 3 matrix

In [92]:
np.random.seed(0)
print(generate_random_vectors(3, 5))

[[ 1.76405235  0.40015721  0.97873798]
 [ 2.2408932   1.86755799 -0.97727788]
 [ 0.95008842 -0.15135721 -0.10321885]
 [ 0.4105985   0.14404357  1.45427351]
 [ 0.76103773  0.12167502  0.44386323]]


Generate 16 random vectors of dimension 547979, the same dimensionality as our vocubulary size (547979)

In [93]:
np.random.seed(0)
random_vectors = generate_random_vectors(16, 547979)
print(random_vectors.shape)

(547979, 16)


#### Partition data points into bins

Check if doc 0 belongs to bin 1

In [94]:
doc = corpus[0, :]
print((doc.dot(random_vectors[:, 0])) >= 0)

[ True]


Check if doc 1 belongs to bin 2

In [95]:
print((doc.dot(random_vectors[:, 1])) >= 0)

[ True]


In [96]:
print(doc.dot(random_vectors) >= 0)

[[ True  True False False False  True  True False  True  True  True False
  False  True False  True]]


In [97]:
print(np.array(doc.dot(random_vectors) >= 0, dtype=int))

[[1 1 0 0 0 1 1 0 1 1 1 0 0 1 0 1]]


All documents that obtain exactly this vector will be assigned to the same bin.

In [98]:
print(corpus[0:2].dot(random_vectors) >= 0) # compute bit indices of first two documents
print(corpus.dot(random_vectors) >= 0) # compute bit indices of ALL documents

[[ True  True False False False  True  True False  True  True  True False
  False  True False  True]
 [ True False False False  True  True False  True  True False  True False
   True False False  True]]
[[ True  True False ...,  True False  True]
 [ True False False ..., False False  True]
 [False  True False ...,  True False  True]
 ..., 
 [ True  True False ...,  True  True  True]
 [False  True  True ...,  True False  True]
 [ True False  True ..., False False  True]]


To make it convenient to refer to individual bins, we convert each binary bin index into a single integer:<br>
For example:<br>
[0,0,0,0,0,0,0,0,0,0,0,0]   => 0<br>
[0,0,0,0,0,0,0,0,0,0,0,1]   => 1<br>
[1,1,1,1,1,1,1,1,1,1,1,1]   => 65535 (= 2^16-1)<br>

To do that, we just need to compute the dot product between the document vector and the vector consisting of powers of 2

In [99]:
doc = corpus[0, :]
index_bits = (doc.dot(random_vectors) >= 0)
power_of_two = (1 << np.arange(15, -1, -1))
print(index_bits.dot(power_of_two))

[50917]


In [100]:
index_bits = corpus.dot(random_vectors) > 0
print(index_bits.dot(power_of_two))

[50917 36265 19365 ..., 52983 27589 41449]


In [101]:
def train_lsh(data, num_vector=16, seed=None):
    dim = data.shape[1]
    if seed is not None:
        np.random.seed(seed)
    random_vectors = generate_random_vectors(num_vector, dim)
    
    powers_of_two = 1 << np.arange(num_vector-1, -1, -1)
    
    table = {}
    
    bin_index_bits = (data.dot(random_vectors) >= 0)
    
    bin_indices = bin_index_bits.dot(powers_of_two)
    
    for data_index, bin_index in enumerate(bin_indices):
        table.setdefault(bin_index, []).append(data_index)
        
    model = {'data': data,
             'bin_index_bits': bin_index_bits,
             'bin_indices': bin_indices,
             'table': table,
             'random_vectors': random_vectors,
             'num_vectors': num_vector}
    
    return model

In [102]:
model = train_lsh(corpus, num_vector=16, seed=143)

Check

In [103]:
table = model['table']
if 0 in table and table[0] == [39583] and 143 in table and table[143] == [19693, 28277, 29776, 30399]:
    print('Passed!')
else:
    print('Check your code.')

Passed!


#### OK! Now we have the model, let's rock

In [104]:
print(wiki[wiki['name'] == 'Barack Obama'])

                                              URI          name  \
35817  <http://dbpedia.org/resource/Barack_Obama>  Barack Obama   

                                                    text  
35817  barack hussein obama ii brk husen bm born augu...  


##### Quiz Question. What is the document id of Barack Obama's article?<br>

35817

##### Quiz Question. Which bin contains Barack Obama's article? Enter its integer index.

In [105]:
print(model['bin_indices'][35817])

50194


##### Quiz Question. Examine the bit representations of the bins containing Barack Obama and Joe Biden. In how many places do they agree?

In [106]:
row_number = wiki[wiki['name'] == 'Joe Biden'].index.values

In [107]:
model['bin_index_bits'][row_number] == model['bin_index_bits'][35817]

array([[ True, False,  True,  True,  True,  True,  True,  True,  True,
         True,  True, False,  True,  True,  True,  True]], dtype=bool)

Compare the result with a former British diplomat

In [108]:
print(wiki[wiki['name']=='Wynn Normington Hugh-Jones'])

print(np.array(model['bin_index_bits'][22745], dtype=int)) # list of 0/1's
print(model['bin_index_bits'][35817] == model['bin_index_bits'][22745])

                                                     URI  \
22745  <http://dbpedia.org/resource/Wynn_Normington_H...   

                             name  \
22745  Wynn Normington Hugh-Jones   

                                                    text  
22745  sir wynn normington hughjones kb sometimes kno...  
[0 0 0 1 0 0 1 0 0 0 1 1 0 1 0 0]
[False False  True False  True False False  True  True  True False  True
  True False False  True]


Let's look at which documents are in the same bin as the Barack Obama article.

In [109]:
print(model['table'][model['bin_indices'][35817]])

[21426, 35817, 39426, 50261, 53937]


In [112]:
doc_ids = model['table'][model['bin_indices'][35817]].copy()
doc_ids.remove(35817)

docs = wiki.loc[doc_ids]

Measure similarity

In [119]:
from sklearn import preprocessing

In [139]:
def norm(x):
    sum_sq=x.dot(x.T)
    norm=np.sqrt(sum_sq)
    return(norm)

def cosine_distance(x, y):
    xy = x.dot(y.T)
    dist = xy/(norm(x) * norm(y))
    return 1-dist[0, 0]

obama_tf_idf = corpus[35817,:]
biden_tf_idf = corpus[24478,:]

print('================= Cosine distance from Barack Obama')
print('Barack Obama - {0:24s}: {1:f}'.format('Joe Biden', cosine_distance(obama_tf_idf, biden_tf_idf)))
for doc_id in doc_ids:
    doc_tf_idf = corpus[doc_id,:]
    print('Barack Obama - {0:24s}: {1:f}'.format(wiki.loc[doc_id]['name'],
                                                 cosine_distance(obama_tf_idf, doc_tf_idf)))

================= Cosine distance from Barack Obama
Barack Obama - Joe Biden               : 0.703139
Barack Obama - Mark Boulware           : 0.950867
Barack Obama - John Wells (politician) : 0.975966
Barack Obama - Francis Longstaff       : 0.978256
Barack Obama - Madurai T. Srinivasan   : 0.993092
